In [ ]:
!pip install ultralytics --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.2 MB/s eta 0:00:00


In [ ]:
!pip install torchmetrics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 56.7 MB/s eta 0:00:00


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import os
import shutil
from ultralytics import YOLO
from pathlib import Path
from google.colab import drive
import pandas as pd
import torch
from ultralytics.cfg import get_cfg

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA Available: True
Device Name: NVIDIA L4


In [ ]:
dataset_path = "/content/drive/MyDrive/UMich/PR/EuroPallet"
data_yaml = os.path.join(dataset_path, "data.yaml")

output_dir = "/content/drive/MyDrive/UMich/PR/EuroPallet/results"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
run_dir = "/content/drive/MyDrive/UMich/PR/EuroPallet/results"
weights_dir = os.path.join(run_dir, "weights")
log_path = os.path.join(run_dir, "results.csv")
checkpoint_dir = os.path.join(run_dir, "models")
os.makedirs(checkpoint_dir, exist_ok=True)

In [ ]:
class SaveAllBestCheckpoints:
    def __init__(self, run_dir):
        self.best_metric = 0
        self.checkpoint_dir = os.path.join(run_dir, "models")
        os.makedirs(self.checkpoint_dir, exist_ok=True)

    def __call__(self, trainer):
        epoch = trainer.epoch + 1
        metrics = trainer.metrics

        if metrics is not None and 'metrics/mAP50(B)' in metrics:
            current_map = metrics.get('metrics/mAP50(B)', 0)
            if current_map > self.best_metric:
                self.best_metric = current_map

                src = os.path.join(trainer.save_dir, "weights", "best.pt")
                filename = f"best_epoch{epoch}_mAP{current_map:.3f}.pt"
                dest = os.path.join(self.checkpoint_dir, filename)

                shutil.copy(src, dest)
                print(f"📌 Checkpoint saved: {filename}")

In [ ]:
model = YOLO("yolov8s.pt")

save_checkpoints_callback = SaveAllBestCheckpoints(run_dir)

model.add_callback("on_train_epoch_end", save_checkpoints_callback)

results = model.train(
    data=data_yaml,
    epochs=100,
    imgsz=640,
    project="/content/drive/MyDrive/UMich/PR/EuroPallet",
    name="results",
    val=True,
    save=True,
    patience=10,
    lr0=0.01,
    optimizer='SGD',
    device=0,
    batch=16,
    workers=2,
    verbose=True,
    plots=True,
    resume=False
)

100%|██████████| 21.5M/21.5M [00:00<00:00, 214MB/s]


Ultralytics 8.3.110 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/UMich/PR/EuroPallet/data.yaml, epochs=100, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=/content/drive/MyDrive/UMich/PR/EuroPallet, name=results2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False

100%|██████████| 755k/755k [00:00<00:00, 96.2MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 334MB/s]


...

     86/100      9.77G      1.418     0.7219      1.094        356        640: 100%|██████████| 264/264 [00:47<00:00,  5.59it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:03<00:00,  4.30it/s]

                   all        525      10932      0.831      0.745      0.823      0.432
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 76, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



86 epochs completed in 1.258 hours.
Optimizer stripped from /content/drive/MyDrive/UMich/PR/EuroPallet/results2/weights/last.pt, 22.5MB
Optimizer stripped from /content/drive/MyDrive/UMich/PR/EuroPallet/results2/weights/best.pt, 22.5MB

Validating /content/drive/MyDrive/UMich/PR/EuroPallet/results2/weights/best.pt...
Ultralytics 8.3.110 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:05<00:00,  3.03it/s]


                   all        525      10932       0.83       0.75      0.823      0.433
Speed: 0.2ms preprocess, 2.5ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/drive/MyDrive/UMich/PR/EuroPallet/results2


In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/drive/MyDrive/UMich/PR/EuroPallet/results/runs"